In [1]:
"""
You can run either this notebook locally (if you have all the dependencies and a GPU) or on Google Colab.

Instructions for setting up Colab are as follows:
1. Open a new Python 3 notebook.
2. Import this notebook from GitHub (File -> Upload Notebook -> "GITHUB" tab -> copy/paste GitHub URL)
3. Connect to an instance with a GPU (Runtime -> Change runtime type -> select "GPU" for hardware accelerator)
4. Run this cell to set up dependencies.
5. Restart the runtime (Runtime -> Restart Runtime) for any upgraded packages to take effect


NOTE: User is responsible for checking the content of datasets and the applicable licenses and determining if suitable for the intended use.
"""
# If you're using Google Colab and not running locally, run this cell.
import os

# Install dependencies
!pip install wget
# !apt install sox libsndfile1 ffmpeg
!pip install text-unidecode
!pip install matplotlib>=3.3.2

## Install NeMo
# BRANCH = 'r2.3.0'
# !python -m pip install git+https://github.com/NVIDIA/NeMo.git@$BRANCH#egg=nemo_toolkit[all]

## Grab the config we'll use in this example
# !mkdir configs
# !wget -P configs/ https://raw.githubusercontent.com/NVIDIA/NeMo/$BRANCH/examples/asr/conf/contextnet_rnnt/contextnet_rnnt.yaml


In [2]:
# In a conda environment, you would use the following command
# Update Numba to > 0.53
# conda install -c conda-forge numba
# or
# conda update -c conda-forge numba

# For pip based environments,
# Update Numba to > 0.53
# !pip install --upgrade numba

# Automatic Speech Recognition with Transducer Models

This notebook is a basic tutorial for creating a Transducer ASR model and then training it on a small dataset (AN4). It includes discussion relevant to reducing memory issues when training such models and demonstrates how to change the decoding strategy after training. Finally, it also provides a brief glimpse of extracting alignment information from a trained Transducer model.

As we will see in this tutorial, apart from the differences in the config and the class used to instantiate the model, nearly all steps are precisely similar to any CTC-based model training. Many concepts such as data loader setup, optimization setup, pre-trained checkpoint weight loading will be nearly identical between CTC and Transducer models.

In essence, NeMo makes it seamless to take a config for a CTC ASR model, add in a few components related to Transducers (often without any modifications) and use a different class to instantiate a Transducer model!

--------

**Note**: It is assumed that the previous tutorial - "Intro-to-Transducers" has been reviewed, and there is some familiarity with the config components of transducer models.


# Preparing the dataset

In this tutorial, we will be utilizing the `AN4`dataset - also known as the Alphanumeric dataset, which was collected and published by Carnegie Mellon University. It consists of recordings of people spelling out addresses, names, telephone numbers, etc., one letter or number at a time and their corresponding transcripts. We choose to use AN4 for this tutorial because it is relatively small, with 948 training and 130 test utterances, and so it trains quickly.

Let's first download the preparation script from NeMo's scripts directory -

In [3]:
import os

if not os.path.exists("scripts/"):
  os.makedirs("scripts")

# if not os.path.exists("scripts/process_an4_data.py"):
#   !wget -P scripts/ https://raw.githubusercontent.com/NVIDIA/NeMo/$BRANCH/scripts/dataset_processing/process_an4_data.py

------

Download and prepare the two subsets of `AN 4`

In [4]:
import wget
import tarfile
import subprocess
import glob

data_dir = "datasets"

if not os.path.exists(data_dir):
  os.makedirs(data_dir)

# Download the dataset. This will take a few moments...
print("******")
if not os.path.exists(data_dir + '/an4_sphere.tar.gz'):
    an4_url = 'https://dldata-public.s3.us-east-2.amazonaws.com/an4_sphere.tar.gz'
    an4_path = wget.download(an4_url, data_dir)
    print(f"Dataset downloaded at: {an4_path}")
else:
    print("Tarfile already exists.")
    an4_path = data_dir + '/an4_sphere.tar.gz'


if not os.path.exists(data_dir + '/an4/'):
    # Untar and convert .sph to .wav (using sox)
    tar = tarfile.open(an4_path)
    tar.extractall(path=data_dir)

    print("Converting .sph to .wav...")
    sph_list = glob.glob(data_dir + '/an4/**/*.sph', recursive=True)
    for sph_path in sph_list:
        wav_path = sph_path[:-4] + '.wav'
        cmd = ["sox", sph_path, wav_path]
        subprocess.run(cmd)

print("Finished conversion.\n******")

******
Tarfile already exists.
Finished conversion.
******


In [5]:
# --- Building Manifest Files --- #
import json
import librosa

# Function to build a manifest
def build_manifest(transcripts_path, manifest_path, wav_path):
    with open(transcripts_path, 'r') as fin:
        with open(manifest_path, 'w') as fout:
            for line in fin:
                # Lines look like this:
                # <s> transcript </s> (fileID)
                transcript = line[: line.find('(')-1].lower()
                transcript = transcript.replace('<s>', '').replace('</s>', '')
                transcript = transcript.strip()

                file_id = line[line.find('(')+1 : -2]  # e.g. "cen4-fash-b"
                audio_path = os.path.join(
                    data_dir, wav_path,
                    file_id[file_id.find('-')+1 : file_id.rfind('-')],
                    file_id + '.wav')

                duration = librosa.core.get_duration(path=audio_path)

                # Write the metadata to the manifest
                metadata = {
                    "audio_filepath": audio_path,
                    "duration": duration,
                    "text": transcript
                }
                json.dump(metadata, fout)
                fout.write('\n')

# Building Manifests
print("******")
train_transcripts = os.path.join(data_dir, 'an4/etc/an4_train.transcription')
train_manifest = os.path.join(data_dir, 'an4/train_manifest.json')
if not os.path.isfile(train_manifest):
    build_manifest(train_transcripts, train_manifest, 'an4/wav/an4_clstk')
    print("Training manifest created.")

test_transcripts = os.path.join(data_dir, 'an4/etc/an4_test.transcription')
test_manifest = os.path.join(data_dir, 'an4/test_manifest.json')
if not os.path.isfile(test_manifest):
    build_manifest(test_transcripts, test_manifest, 'an4/wav/an4test_clstk')
    print("Test manifest created.")
print("***Done***")
# Manifest filepaths
TRAIN_MANIFEST = train_manifest
TEST_MANIFEST = test_manifest

******
***Done***


## Preparing the tokenizer

Now that we have a dataset ready, we need to decide whether to use a character-based model or a sub-word-based model. For completeness' sake, we will use a tokenizer based model so that we can leverage a modern encoder architecture like ContextNet or Conformer-T.

In [6]:
# if not os.path.exists("scripts/process_asr_text_tokenizer.py"):
#   !wget -P scripts/ https://raw.githubusercontent.com/NVIDIA/NeMo/$BRANCH/scripts/tokenizers/process_asr_text_tokenizer.py

-----

Since the dataset is tiny, we can use a small SentencePiece based tokenizer. We always delete the tokenizer directory so any changes to the manifest files are always replicated in the tokenizer.

In [7]:
VOCAB_SIZE = 64  # can be any value above 29
TOKENIZER_TYPE = "spe"  # can be wpe or spe
SPE_TYPE = "bpe"  # can be bpe or unigram

# ------------------------------------------------------------------- #
!rm -r tokenizers/

if not os.path.exists("tokenizers"):
  os.makedirs("tokenizers")

!python scripts/process_asr_text_tokenizer.py \
   --manifest=$TRAIN_MANIFEST \
   --data_root="tokenizers" \
   --tokenizer=$TOKENIZER_TYPE \
   --spe_type=$SPE_TYPE \
   --no_lower_case \
   --log \
   --vocab_size=$VOCAB_SIZE

INFO:root:Finished extracting manifest : datasets/an4/train_manifest.json
INFO:root:Finished extracting all manifests ! Number of sentences : 948
[NeMo I 2025-07-18 15:02:35 sentencepiece_tokenizer:425] Processing tokenizers/text_corpus/document.txt and store at tokenizers/tokenizer_spe_bpe_v64
sentencepiece_trainer.cc(178) LOG(INFO) Running command: --input=tokenizers/text_corpus/document.txt --model_prefix=tokenizers/tokenizer_spe_bpe_v64/tokenizer --vocab_size=64 --shuffle_input_sentence=true --hard_vocab_limit=false --model_type=bpe --character_coverage=1.0 --bos_id=-1 --eos_id=-1 --remove_extra_whitespaces=false
sentencepiece_trainer.cc(78) LOG(INFO) Starts training with : 
trainer_spec {
  input: tokenizers/text_corpus/document.txt
  input_format: 
  model_prefix: tokenizers/tokenizer_spe_bpe_v64/tokenizer
  model_type: BPE
  vocab_size: 64
  self_test_sample_size: 0
  character_coverage: 1
  input_sentence_size: 0
  shuffle_input_sentence: 1
  seed_sentencepiece_size: 1000000
  

In [8]:
# Tokenizer path
if TOKENIZER_TYPE == 'spe':
  TOKENIZER = os.path.join("tokenizers", f"tokenizer_spe_{SPE_TYPE}_v{VOCAB_SIZE}")
  TOKENIZER_TYPE_CFG = "bpe"
else:
  TOKENIZER = os.path.join("tokenizers", f"tokenizer_wpe_v{VOCAB_SIZE}")
  TOKENIZER_TYPE_CFG = "wpe"

# Preparing a Transducer Model

Now that we have the dataset and tokenizer prepared, let us begin by setting up the config of the Transducer model! In this tutorial, we will build a slightly modified ContextNet architecture (which is obtained from the paper [ContextNet: Improving Convolutional Neural Networks for Automatic Speech Recognition with Global Context](https://arxiv.org/abs/2005.03191)).

We can note that many of the steps here are identical to the setup of a CTC model!


## Prepare the config

For a dataset such as AN4, we do not need such a deep model. In fact, the depth of this model will cause much slower convergence on a small dataset, which would require far too long to train on Colab.

In order to speed up training for this demo, we will take only the first five blocks of ContextNet, and discard the rest - and we can do this directly from the config.

**Note**: On any realistic dataset (say Librispeech) this step would hurt the model's accuracy significantly. It is being done only to reduce the time spent waiting for training to finish on Colab.

In [9]:
from omegaconf import OmegaConf, open_dict

# config = OmegaConf.load("configs/contextnet_rnnt.yaml")
config = OmegaConf.load("configs/fast-conformer_transducer_bpe.yaml")

-----

Here, we will slice off the first five blocks from the Jasper block (used to build ContextNet). Setting the config with this subset will create a stride 2x model with just five blocks.

We will also explicitly state that the last block dimension must be obtained from `model.model_defaults.enc_hidden` inside the config.

In [10]:
# config.model.encoder.jasper = config.model.encoder.jasper[:5]
# config.model.encoder.jasper[-1].filters = '${model.model_defaults.enc_hidden}'
config.model.encoder.n_layers = 3
config.model.encoder.d_model = 64
config.model.train_ds.max_duration = 5
# config.model.encoder.conv_kernel_size = 18

-------

Next, set up the data loaders of the config for the ContextNet model.

In [11]:
# print out the train and validation configs to know what needs to be changed
print(OmegaConf.to_yaml(config.model.train_ds))

manifest_filepath: ???
sample_rate: ${model.sample_rate}
batch_size: 16
shuffle: true
num_workers: 8
pin_memory: true
max_duration: 5
min_duration: 0.1
is_tarred: false
tarred_audio_filepaths: null
shuffle_n: 2048
bucketing_strategy: fully_randomized
bucketing_batch_size: null



-------

We can note that the config here is nearly identical to the CTC ASR model configs! So let us take the same steps here to update the configs.

In [12]:
config.model.train_ds.manifest_filepath = TRAIN_MANIFEST
config.model.validation_ds.manifest_filepath = TEST_MANIFEST
config.model.test_ds.manifest_filepath = TEST_MANIFEST

------

Next, we need to setup the tokenizer section of the config

In [13]:
print(OmegaConf.to_yaml(config.model.tokenizer))

dir: ???
type: bpe



In [14]:
config.model.tokenizer.dir = TOKENIZER
config.model.tokenizer.type = TOKENIZER_TYPE_CFG

------
Now, we can update the optimization and augmentation for this dataset in order to converge to some reasonable score within a short training run.

In [15]:
print(OmegaConf.to_yaml(config.model.optim))

name: novograd
lr: 0.05
betas:
- 0.9
- 0.0
weight_decay: 0.001
sched:
  name: CosineAnnealing
  warmup_steps: 5000
  warmup_ratio: null
  min_lr: 1.0e-06
  last_epoch: -1



In [16]:
# Finally, let's remove logging of samples and the warmup since the dataset is small (similar to CTC models)
config.model.log_prediction = False
config.model.optim.sched.warmup_steps = None

------

Next, we remove the spec augment that is provided by default for ContextNet. While additional augmentation would surely help training, it would require longer training to see significant benefits.

In [17]:
print(OmegaConf.to_yaml(config.model.spec_augment))

_target_: nemo.collections.asr.modules.SpectrogramAugmentation
freq_masks: 2
time_masks: 10
freq_width: 27
time_width: 0.05



In [18]:
config.model.spec_augment.freq_masks = 0
config.model.spec_augment.time_masks = 0

------

... We are now almost done! Most of the updates to a Transducer config are nearly the same as any CTC model.

## Fused Batch during training and evaluation

We discussed in the previous tutorial (Intro-to-Transducers) the significant memory cost of the Transducer Joint calculation during training. We also discussed that NeMo provides a simple yet effective method to nearly sidestep this limitation. We can now dive deeper into understanding what precisely NeMo's Transducer framework will do to alleviate this memory consumption issue.

The following sub-cells are **voluntary** and valuable for understanding the cause, effect, and resolution of memory issues in Transducer models. The content can be skipped if one is familiar with the topic, and it is only required to use the `fused batch step`.

## Transducer Memory reduction with Fused Batch step

The following few cells explain why memory is an issue when training Transducer models and how NeMo tackles the issue with its Fused Batch step.

The material can be read for a thorough understanding, otherwise, it can be skipped.

### Diving deeper into the memory costs of Transducer Joint
-------

One of the significant limitations of Transducers is the exorbitant memory cost of computing the Joint module. The Joint module is comprised of two steps.

1) Projecting the Acoustic and Transcription feature dimensions to some standard hidden dimension (specified by `model.model_defaults.joint_hidden`)

2) Projecting this intermediate hidden dimension to the final vocabulary space to obtain the transcription.

Take the following example.

**BS**=32 ; **T** (after **2x** stride) = 800, **U** (with character encoding) = 400-450 tokens, Vocabulary size  **V** = 28 (26 alphabet chars, space and apostrophe). Let the hidden dimension of the Joint model be 640 (Most Google Transducer papers use hidden dimension of 640).

$ Memory \, (Hidden, \, gb) = 32 \times 800 \times 450 \times 640 \times 4 = 29.49 $ gigabytes (4 bytes per float).

$ Memory \, (Joint, \, gb) = 32 \times 800 \times 450 \times 28 \times 4 = 1.290 $ gigabytes (4 bytes per float)

-----

**NOTE**: This is just for the forward pass! We need to double this memory to store gradients! This much memory is also just for the Joint model **alone**. Far more memory is required for the Prediction model as well as the large Acoustic model itself and its gradients!

Even with mixed precision, that's $\sim 30$ GB of GPU RAM for just 1 part of the network + its gradients.

---------

### Simple methods to reduce memory consumption

------

The easiest way to reduce memory consumption is to perform more downsampling in the acoustic model and use sub-word tokenization of the text to reduce the length of the target sequence.

**BS**=32 ; **T** (after **8x** stride) = 200, **U** (with sub-word encoding) = 100-180 tokens, Vocabulary size  **V** = 1024.

$ Memory \, (Hidden, \, gb) = 32 \times 200 \times 150 \times 640 \times 4 = 2.45 $ gigabytes (4 bytes per float).

$ Memory \, (Joint, \, gb) = 32 \times 200 \times 150 \times 1024 \times 4 = 3.93 $ gigabytes (4 bytes per float)

-----

Using Automatic Mixed Precision, we expend just around 6-7 GB of GPU RAM on the Joint + its gradient.

The above memory cost is much more tractable - but we generally want larger and larger acoustic models. It is consistently the easiest way to improve transcription accuracy. So that means on a limited 32 GB GPU, we have to partition 7 GB just for the Joint and remaining memory allocated between Transcription + Acoustic Models.



### Fused Transcription-Joint-Loss-WER (also called Batch Splitting)
----------

The fundamental problem is that the joint tensor grows in size when `[T x U]` grows in size. This growth in memory cost is due to many reasons - either by model construction (downsampling) or the choice of dataset preprocessing (character tokenization vs. sub-word tokenization).

Another dimension that NeMo can control is **batch**. Due to how we batch our samples, small and large samples all get clumped together into a single batch. So even though the individual samples are not all as long as the maximum length of T and U in that batch, when a batch of such samples is constructed, it will consume a significant amount of memory for the sake of compute efficiency.

So as is always the case - **trade-off compute speed for memory savings**.

------

The fused operation goes as follows :

1) Forward the entire acoustic model in a single pass. (Use global batch size here for acoustic model - found in `model.*_ds.batch_size`)

2) Split the Acoustic Model's logits by `fused_batch_size` and loop over these sub-batches.

3) Construct a sub-batch of same `fused_batch_size` for the Prediction model. Now the target sequence length is $U_{sub-batch} < U$.

4) Feed this $U_{sub-batch}$ into the Joint model, along with a sub-batch from the Acoustic model (with $T_{sub-batch} < T$). Remember, we only have to slice off a part of the acoustic model here since we have the full batch of samples $(B, T, D)$ from the acoustic model.

5) Performing steps (3) and (4) yields $T_{sub-batch}$ and $U_{sub-batch}$. Perform sub-batch joint step - costing an intermediate $(B, T_{sub-batch}, U_{sub-batch}, V)$ in memory.

6) Compute loss on sub-batch and preserve in a list to be later concatenated.

7) Compute sub-batch metrics (such as Character / Word Error Rate) using the above Joint tensor and sub-batch of ground truth labels. Preserve the scores to be averaged across the entire batch later.

8) Delete the sub-batch joint matrix  $(B, T_{sub-batch}, U_{sub-batch}, V)$. Only gradients from .backward() are preserved now in the computation graph.

9) Repeat steps (3) - (8) until all sub-batches are consumed.

10) Cleanup step. Compute full batch WER and log. Concatenate loss list and pass to PTL to compute the equivalent of the original (full batch) Joint step. Delete ancillary objects necessary for sub-batching.


## Setting up Fused Batch step in a Transducer Config

After all that discussion above, let us look at how to enable that entire pipeline in NeMo.

As we can note below, it takes precisely two changes in the config to enable the fused batch step:

In [19]:
print(OmegaConf.to_yaml(config.model.joint))

_target_: nemo.collections.asr.modules.RNNTJoint
log_softmax: null
preserve_memory: false
fuse_loss_wer: true
fused_batch_size: 4
jointnet:
  joint_hidden: ${model.model_defaults.joint_hidden}
  activation: relu
  dropout: 0.2



In [20]:
# Two lines to enable the fused batch step
config.model.joint.fuse_loss_wer = True
config.model.joint.fused_batch_size = 16  # this can be any value (preferably less than model.*_ds.batch_size)

In [21]:
# We will also reduce the hidden dimension of the joint and the prediction networks to preserve some memory
config.model.model_defaults.pred_hidden = 64
config.model.model_defaults.joint_hidden = 64

--------

Finally, since the dataset is tiny, we do not need an enormous model (the default is roughly 40 M parameters!).

In [22]:
# Use just 128 filters across the model to speed up training and reduce parameter count
config.model.model_defaults.filters = 128

## Initialize a Transducer ASR Model

Finally, let us create a Transducer model, which is as easy as changing a line of import if you already have a script to create CTC models. We will use a small model since the dataset is just 5 hours of speech.

------

Setup a Pytorch Lightning Trainer:

In [23]:
import torch
from lightning.pytorch import Trainer

if torch.cuda.is_available():
  accelerator = 'gpu'
else:
  accelerator = 'gpu'

EPOCHS = 10

# Initialize a Trainer for the Transducer model
trainer = Trainer(devices=1, accelerator=accelerator, max_epochs=EPOCHS,
                  enable_checkpointing=False, logger=False,
                  log_every_n_steps=1, check_val_every_n_epoch=1)

/home/ubuntu/miniconda3/envs/nemo/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


In [24]:
# Import the Transducer Model
import nemo.collections.asr as nemo_asr

In [25]:
# Build the model
model = nemo_asr.models.EncDecRNNTBPEModel(cfg=config.model, trainer=trainer)

[NeMo I 2025-07-18 15:02:42 mixins:181] Tokenizer SentencePieceTokenizer initialized with 64 tokens
[NeMo I 2025-07-18 15:02:42 collections:201] Dataset loaded with 910 files totalling 0.65 hours
[NeMo I 2025-07-18 15:02:42 collections:202] 38 files were filtered totalling 0.06 hours


[NeMo I 2025-07-18 15:02:42 collections:201] Dataset loaded with 130 files totalling 0.10 hours
[NeMo I 2025-07-18 15:02:42 collections:202] 0 files were filtered totalling 0.00 hours
[NeMo I 2025-07-18 15:02:42 collections:201] Dataset loaded with 130 files totalling 0.10 hours
[NeMo I 2025-07-18 15:02:42 collections:202] 0 files were filtered totalling 0.00 hours
[NeMo I 2025-07-18 15:02:43 features:305] PADDING: 0
[NeMo I 2025-07-18 15:02:43 rnnt_models:226] Using RNNT Loss : warprnnt_numba
    Loss warprnnt_numba_kwargs: {'fastemit_lambda': 0.001, 'clamp': -1.0}
[NeMo I 2025-07-18 15:02:43 rnnt_models:226] Using RNNT Loss : warprnnt_numba
    Loss warprnnt_numba_kwargs: {'fastemit_lambda': 0.001, 'clamp': -1.0}
[NeMo I 2025-07-18 15:02:43 rnnt_models:226] Using RNNT Loss : warprnnt_numba
    Loss warprnnt_numba_kwargs: {'fastemit_lambda': 0.001, 'clamp': -1.0}


In [26]:
model.summarize()

  | Name              | Type                              | Params | Mode 
--------------------------------------------------------------------------------
0 | preprocessor      | AudioToMelSpectrogramPreprocessor | 0      | train
1 | encoder           | ConformerEncoder                  | 605 K  | train
2 | decoder           | RNNTDecoder                       | 37.4 K | train
3 | joint             | RNNTJoint                         | 12.5 K | train
4 | loss              | RNNTLoss                          | 0      | train
5 | spec_augmentation | SpectrogramAugmentation           | 0      | train
6 | wer               | WER                               | 0      | train
--------------------------------------------------------------------------------
655 K     Trainable params
0         Non-trainable params
655 K     Total params
2.624     Total estimated model params size (MB)
117       Modules in train mode
0         Modules in eval mode

------

We now have a Transducer model ready to be trained!

## (Optional) Partially loading pre-trained weights from another model

An interesting point to note about Transducer models - the Acoustic model config (and therefore the Acoustic model itself) can be shared between CTC and Transducer models.

This means that we can initialize the weights of a Transducer's Acoustic model with weights from a pre-trained CTC encoder model.

------

**Note**: This step is optional and not necessary at all to train a Transducer model. Below, we show the steps that we would take if we wanted to do this, however as the loaded model has different kernel sizes compared to the current model, the checkpoint cannot be loaded.

In [27]:
# Load a small CTC model
# ctc_model = nemo_asr.models.EncDecCTCModelBPE.from_pretrained("stt_en_citrinet_256", map_location='cpu')
# trans_model = nemo_asr.models.EncDecRNNTBPEModel.from_pretrained("stt_en_fastconformer_transducer_large", map_location='cpu')

------

Then load the state dict of the CTC model's encoder into the Transducer model's encoder.

In [28]:
# <<< NOTE: This is only for demonstration ! >>>
# Below cell will fail because the two model's have incompatible kernel sizes in their Conv layers.

# <<< NOTE: Below cell is only shown to illustrate the method >>>
# model.encoder.load_state_dict(trans_model.encoder.state_dict(), strict=True)

# Training on AN4

Now that the model is ready, we can finally train it!

In [29]:
# Prepare NeMo's Experiment manager to handle checkpoint saving and logging for us
from nemo.utils import exp_manager

# Environment variable generally used for multi-node multi-gpu training.
# In notebook environments, this flag is unnecessary and can cause logs of multiple training runs to overwrite each other.
os.environ.pop('NEMO_EXPM_VERSION', None)

exp_config = exp_manager.ExpManagerConfig(
    exp_dir=f'experiments/',
    name=f"Transducer-Model",
    checkpoint_callback_params=exp_manager.CallbackParams(
        monitor="val_wer",
        mode="min",
        always_save_nemo=True,
        save_best_model=True,
    ),
)

exp_config = OmegaConf.structured(exp_config)

logdir = exp_manager.exp_manager(trainer, exp_config)

[NeMo I 2025-07-18 15:02:43 exp_manager:561] ExpManager schema
[NeMo I 2025-07-18 15:02:43 exp_manager:562] {'explicit_log_dir': None, 'exp_dir': None, 'name': None, 'version': None, 'use_datetime_version': True, 'resume_if_exists': False, 'resume_past_end': False, 'resume_ignore_no_checkpoint': False, 'resume_from_checkpoint': None, 'create_tensorboard_logger': True, 'summary_writer_kwargs': None, 'create_wandb_logger': False, 'wandb_logger_kwargs': None, 'create_mlflow_logger': False, 'mlflow_logger_kwargs': {'experiment_name': None, 'tracking_uri': None, 'tags': None, 'save_dir': './mlruns', 'prefix': '', 'artifact_location': None, 'run_id': None, 'log_model': False}, 'create_dllogger_logger': False, 'dllogger_logger_kwargs': {'verbose': False, 'stdout': False, 'json_file': './dllogger.json'}, 'create_clearml_logger': False, 'clearml_logger_kwargs': {'project': None, 'task': None, 'connect_pytorch': False, 'model_name': None, 'tags': None, 'log_model': False, 'log_cfg': False, 'log_

In [30]:
try:
  from google import colab
  COLAB_ENV = True
except (ImportError, ModuleNotFoundError):
  COLAB_ENV = False

# Load the TensorBoard notebook extension
if COLAB_ENV:
  %load_ext tensorboard
  %tensorboard --logdir /content/experiments/Transducer-Model/
else:
  print("To use TensorBoard, please use this notebook in a Google Colab environment.")

To use TensorBoard, please use this notebook in a Google Colab environment.


In [31]:
# Release resources prior to training
import gc
gc.collect()

if accelerator == 'gpu':
  torch.cuda.empty_cache()

In [32]:
# Train the model
trainer.fit(model)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


[NeMo I 2025-07-18 15:02:43 modelPT:802] Optimizer config = Novograd (
    Parameter Group 0
        amsgrad: False
        betas: [0.9, 0.0]
        eps: 1e-08
        grad_averaging: False
        lr: 0.05
        weight_decay: 0.001
    )
[NeMo I 2025-07-18 15:02:43 lr_scheduler:950] Scheduler "<nemo.core.optim.lr_scheduler.CosineAnnealing object at 0x75acc87896f0>" 
    will be used during training (effective maximum steps = 570) - 
    Parameters : 
    (warmup_steps: null
    warmup_ratio: null
    min_lr: 1.0e-06
    last_epoch: -1
    max_steps: 570
    )



  | Name              | Type                              | Params | Mode 
--------------------------------------------------------------------------------
0 | preprocessor      | AudioToMelSpectrogramPreprocessor | 0      | train
1 | encoder           | ConformerEncoder                  | 605 K  | train
2 | decoder           | RNNTDecoder                       | 37.4 K | train
3 | joint             | RNNTJoint                         | 12.5 K | train
4 | loss              | RNNTLoss                          | 0      | train
5 | spec_augmentation | SpectrogramAugmentation           | 0      | train
6 | wer               | WER                               | 0      | train
--------------------------------------------------------------------------------
655 K     Trainable params
0         Non-trainable params
655 K     Total params
2.624     Total estimated model params size (MB)
117       Modules in train mode
0         Modules in eval mode


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

[NeMo I 2025-07-18 15:02:43 optional_cuda_graphs:79] Enabled CUDA graphs for module <class 'nemo.collections.asr.models.rnnt_bpe_models.EncDecRNNTBPEModel'>.decoding.decoding
[NeMo I 2025-07-18 15:02:43 optional_cuda_graphs:79] Enabled CUDA graphs for module <class 'nemo.collections.asr.metrics.wer.WER'>joint._wer.decoding.decoding
Sanity Checking DataLoader 0: 100%|██████████| 2/2 [00:01<00:00,  1.61it/s][NeMo I 2025-07-18 15:02:45 optional_cuda_graphs:53] Disabled CUDA graphs for module <class 'nemo.collections.asr.models.rnnt_bpe_models.EncDecRNNTBPEModel'>.decoding.decoding
[NeMo I 2025-07-18 15:02:45 optional_cuda_graphs:53] Disabled CUDA graphs for module <class 'nemo.collections.asr.metrics.wer.WER'>joint._wer.decoding.decoding
Training: |          | 0/? [00:00<?, ?it/s]                                [NeMo I 2025-07-18 15:02:45 preemption:56] Preemption requires torch distributed to be initialized, disabling preemption
Epoch 0:   0%|          | 0/57 [00:00<?, ?it/s] [NeMo I 202

...

[NeMo I 2025-07-18 15:03:54 nemo_model_checkpoint:546] Checkpoint save for step 570 started at 1752851034.0214164.
Epoch 9: 100%|██████████| 57/57 [00:06<00:00,  8.70it/s, v_num=2-43, train_step_timing in s=0.080][NeMo I 2025-07-18 15:03:54 optional_cuda_graphs:79] Enabled CUDA graphs for module <class 'nemo.collections.asr.models.rnnt_bpe_models.EncDecRNNTBPEModel'>.decoding.decoding
[NeMo I 2025-07-18 15:03:54 optional_cuda_graphs:79] Enabled CUDA graphs for module <class 'nemo.collections.asr.metrics.wer.WER'>joint._wer.decoding.decoding


`Trainer.fit` stopped: `max_epochs=10` reached.


Epoch 9: 100%|██████████| 57/57 [00:06<00:00,  8.70it/s, v_num=2-43, train_step_timing in s=0.080]
[NeMo I 2025-07-18 15:03:54 nemo_model_checkpoint:546] Checkpoint save for step 570 started at 1752851034.1266387.


Restoring states from the checkpoint path at /home/ubuntu/nvidia_nemo/tutorials/asr/experiments/Transducer-Model/2025-07-18_15-02-43/checkpoints/Transducer-Model--val_wer=1.0000-epoch=0.ckpt
Restored all states from the checkpoint at /home/ubuntu/nvidia_nemo/tutorials/asr/experiments/Transducer-Model/2025-07-18_15-02-43/checkpoints/Transducer-Model--val_wer=1.0000-epoch=0.ckpt


[NeMo I 2025-07-18 15:03:54 nemo_model_checkpoint:316] /home/ubuntu/nvidia_nemo/tutorials/asr/experiments/Transducer-Model/2025-07-18_15-02-43/checkpoints/Transducer-Model.nemo already exists, moving existing checkpoint to /home/ubuntu/nvidia_nemo/tutorials/asr/experiments/Transducer-Model/2025-07-18_15-02-43/checkpoints/Transducer-Model-v1.nemo
[NeMo I 2025-07-18 15:03:54 nemo_model_checkpoint:288] Removing old .nemo backup /home/ubuntu/nvidia_nemo/tutorials/asr/experiments/Transducer-Model/2025-07-18_15-02-43/checkpoints/Transducer-Model-v1.nemo


-------

Lets check what the final performance on the test set.

In [33]:
trainer.test(model)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing: |          | 0/? [00:00<?, ?it/s][NeMo I 2025-07-18 15:03:54 optional_cuda_graphs:79] Enabled CUDA graphs for module <class 'nemo.collections.asr.models.rnnt_bpe_models.EncDecRNNTBPEModel'>.decoding.decoding
[NeMo I 2025-07-18 15:03:54 optional_cuda_graphs:79] Enabled CUDA graphs for module <class 'nemo.collections.asr.metrics.wer.WER'>joint._wer.decoding.decoding
Testing DataLoader 0:   0%|          | 0/9 [00:00<?, ?it/s]

Testing DataLoader 0: 100%|██████████| 9/9 [00:00<00:00, 12.89it/s]


┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│        global_step        │           57.0            │
│         test_wer          │            1.0            │
└───────────────────────────┴───────────────────────────┘

[{'global_step': 57.0, 'test_wer': 1.0}]

------

The model should obtain some score between 10-12% WER after 50 epochs of training. Quite a good score for just 50 epochs of training a tiny model! Note that these are greedy scores, yet they are pretty strong for such a short training run.

We can further improve these scores by using the internal Prediction network to calculate beam scores.

# Changing the Decoding Strategy

During training, for the sake of efficiency, we were using the `greedy_batch` decoding strategy. However, we might want to perform inference with another method - say, beam search.

NeMo allows changing the decoding strategy easily after the model has been trained.

In [34]:
import copy

decoding_config = copy.deepcopy(config.model.decoding)
print(OmegaConf.to_yaml(decoding_config))

strategy: greedy_batch
greedy:
  max_symbols: 10
beam:
  beam_size: 4
  score_norm: true
  return_best_hypothesis: false
  softmax_temperature: 1.0
  tsd_max_sym_exp: 10
  alsd_max_target_len: 5.0
  maes_num_steps: 2
  maes_prefix_alpha: 1
  maes_expansion_beta: 2
  maes_expansion_gamma: 2.3



In [35]:
# Update the config for the decoding strategy
decoding_config.strategy = "alsd"  # Options are `greedy`, `greedy_batch`, `beam`, `tsd` and `alsd`
decoding_config.beam.beam_size = 4  # Increase beam size for better scores, but it will take much longer for transcription !

In [36]:
# Finally update the model's decoding strategy !
model.change_decoding_strategy(decoding_config)

[NeMo I 2025-07-18 15:03:55 rnnt_models:226] Using RNNT Loss : warprnnt_numba
    Loss warprnnt_numba_kwargs: {'fastemit_lambda': 0.001, 'clamp': -1.0}
[NeMo I 2025-07-18 15:03:55 rnnt_bpe_models:506] Changed decoding strategy to 
    model_type: rnnt
    strategy: alsd
    compute_hypothesis_token_set: false
    preserve_alignments: null
    tdt_include_token_duration: null
    confidence_cfg:
      preserve_frame_confidence: false
      preserve_token_confidence: false
      preserve_word_confidence: false
      exclude_blank: true
      aggregation: min
      tdt_include_duration: false
      method_cfg:
        name: entropy
        entropy_type: tsallis
        alpha: 0.33
        entropy_norm: exp
        temperature: DEPRECATED
    fused_batch_size: null
    compute_timestamps: null
    compute_langs: false
    word_seperator: ' '
    segment_seperators:
    - .
    - '!'
    - '?'
    segment_gap_threshold: null
    rnnt_timestamp_type: all
    greedy:
      max_symbols_per_ste

In [37]:
trainer.test(model)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing DataLoader 0:   0%|          | 0/9 [00:00<?, ?it/s]

Beam search progress:: 100%|██████████| 16/16 [00:04<00:00,  3.65sample/s]


Testing DataLoader 0:  11%|█         | 1/9 [00:05<00:43,  0.18it/s]

Beam search progress:: 100%|██████████| 16/16 [00:04<00:00,  3.97sample/s]


...

Testing DataLoader 0:  89%|████████▉ | 8/9 [00:39<00:04,  0.20it/s]

Beam search progress:: 100%|██████████| 2/2 [00:00<00:00,  3.55sample/s]

Testing DataLoader 0: 100%|██████████| 9/9 [00:40<00:00,  0.22it/s]


┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│        global_step        │           57.0            │
│         test_wer          │            1.0            │
└───────────────────────────┴───────────────────────────┘

[{'global_step': 57.0, 'test_wer': 1.0}]

------

Here, we improved our scores significantly by using the `Alignment-Length Synchronous Decoding` beam search. Feel free to try the other algorithms and compare the speed-accuracy tradeoff!

# (Extra) Extracting Transducer Model Alignments

Transducers are unique in the sense that for each timestep $t \le T$, they can emit multiple target tokens $u_t$. During training, this is represented as the $T \times U$ joint that maps to the vocabulary $V$.

During inference, there is no need to compute the full joint $T \times U$. Instead, after the model predicts the `Transducer Blank` token at the current timestep $t$ while predicting the target token $u_t$, the model will move onto the next acoustic timestep $t + 1$. As such, we can obtain the diagonal alignment of the Transducer model per sample relatively simply.

------

**Note**: While alignments can be calculated for both greedy and beam search - it is non-trivial to incorporate this alignment information for beam decoding. Therefore NeMo only supports extracting alignments during greedy decoding.

-----

Restore model to greedy decoding for alignment calculation

In [38]:
decoding_config.strategy = "greedy_batch"

# Special flag which is generally disabled
# Instruct Greedy Decoders to preserve alignment information during autoregressive decoding
with open_dict(decoding_config):
  decoding_config.preserve_alignments = True
  decoding_config.fused_batch_size = -1  # temporarily stop fused batch during inference.

model.change_decoding_strategy(decoding_config)

[NeMo I 2025-07-18 15:04:36 rnnt_models:226] Using RNNT Loss : warprnnt_numba
    Loss warprnnt_numba_kwargs: {'fastemit_lambda': 0.001, 'clamp': -1.0}
[NeMo I 2025-07-18 15:04:36 rnnt_decoding:792] Joint fused batch size <= 0; Will temporarily disable fused batch step in the Joint.
[NeMo I 2025-07-18 15:04:36 rnnt_bpe_models:506] Changed decoding strategy to 
    model_type: rnnt
    strategy: greedy_batch
    compute_hypothesis_token_set: false
    preserve_alignments: true
    tdt_include_token_duration: null
    confidence_cfg:
      preserve_frame_confidence: false
      preserve_token_confidence: false
      preserve_word_confidence: false
      exclude_blank: true
      aggregation: min
      tdt_include_duration: false
      method_cfg:
        name: entropy
        entropy_type: tsallis
        alpha: 0.33
        entropy_norm: exp
        temperature: DEPRECATED
    fused_batch_size: -1
    compute_timestamps: null
    compute_langs: false
    word_seperator: ' '
    segment_

-------

Set up a test data loader that we will use to obtain the alignments for a single batch.

In [39]:
test_dl = model.test_dataloader()
test_dl = iter(test_dl)
batch = next(test_dl)

device = torch.device('cuda' if accelerator == 'gpu' else 'cpu')

In [40]:
def rnnt_alignments(model, batch):
  model = model.to(device)
  encoded, encoded_len = model.forward(
                        input_signal=batch[0].to(device), input_signal_length=batch[1].to(device)
                    )

  current_hypotheses = model.decoding.rnnt_decoder_predictions_tensor(
                        encoded, encoded_len, return_hypotheses=True
                    )

  del encoded, encoded_len

  # current hypothesis is a tuple of
  # 1) best hypothesis
  # 2) Sorted list of hypothesis (if using beam search); None otherwise
  return current_hypotheses

In [41]:
# Get a batch of hypotheses, as well as a batch of all obtained hypotheses (if beam search is used)
hypotheses = rnnt_alignments(model, batch)

------

Select a sample ID from within the batch to observe the alignment information contained in the Hypothesis.

In [42]:
# Select the sample ID from within the batch
SAMPLE_ID = 0

In [43]:
# Obtain the hypothesis for this sample, as well as some ground truth information about this sample
hypothesis = hypotheses[SAMPLE_ID]
original_sample_len = batch[1][SAMPLE_ID]
ground_truth = batch[2][SAMPLE_ID]

In [44]:
# The Hypothesis object contains a lot of useful information regarding the decoding step.
print(hypothesis)

Hypothesis(score=0.0, y_sequence=tensor([], device='cuda:0', dtype=torch.int64), text='', dec_out=None, dec_state=(tensor([[[-0.0242,  0.0619, -0.0180, -0.0441,  0.0556, -0.0782, -0.0863,
           0.0635,  0.1183, -0.0101,  0.0026, -0.0507,  0.0253, -0.0558,
          -0.0889, -0.0571,  0.0171, -0.0800, -0.0482, -0.1455,  0.0117,
          -0.0232, -0.0399, -0.0543,  0.0270, -0.0389,  0.0133, -0.0069,
           0.0956,  0.0197, -0.0468, -0.0548, -0.0407, -0.0206,  0.0755,
          -0.0332, -0.0749,  0.0274, -0.0700,  0.0365, -0.0645, -0.0275,
          -0.0171,  0.0602, -0.0900, -0.0740, -0.0204,  0.0485,  0.0151,
          -0.1327, -0.1086, -0.0380, -0.0562,  0.0066, -0.1000, -0.0648,
           0.1346,  0.0046, -0.1095,  0.0196,  0.0245, -0.0418, -0.0886,
           0.1872]]]), tensor([[[-0.0467,  0.1197, -0.0336, -0.0851,  0.0929, -0.1497, -0.1713,
           0.1199,  0.2498, -0.0176,  0.0052, -0.1036,  0.0450, -0.0949,
          -0.1640, -0.0953,  0.0325, -0.1525, -0.0942, -0.2

-------

Now, decode the hypothesis and compare it against the ground truth text. Note - this decoded hypothesis is at *sub-word* level for this model. Therefore sub-word tokens such as `_` may be seen here.

In [45]:
decoded_text = hypothesis.text
decoded_hypothesis = model.decoding.decode_ids_to_tokens(hypothesis.y_sequence.cpu().numpy().tolist())
decoded_ground_truth = model.decoding.tokenizer.ids_to_text(ground_truth.cpu().numpy().tolist())

print("Decoded ground truth :", decoded_ground_truth)
print("Decoded hypothesis   :", decoded_text)
print("Decoded hyp tokens   :", decoded_hypothesis)


Decoded ground truth : rubout g m e f three nine
Decoded hypothesis   : 
Decoded hyp tokens   : []


---------

Next we print out the 2-d alignment grid of the RNNT model:

In [46]:
alignments = hypothesis.alignments

# These two values should normally always match
print("Length of alignments (T): ", len(alignments))
print("Length of padded acoustic model after striding : ", int(hypothesis.length))

Length of alignments (T):  51
Length of padded acoustic model after striding :  51


------

Finally, let us calculate the alignment grid. We will de-tokenize the sub-word token if it is a valid index in the vocabulary and use `''` as a placeholder for the `Transducer Blank` token.

Note that each `timestep` here is (roughly) $timestep * total\_stride\_of\_model * preprocessor.window\_stride$ seconds timestamp.

**Note**: You can modify the value of `config.model.loss.warprnnt_numba_kwargs.fastemit_lambda` prior to training and see an impact on final alignment latency!

In [47]:
# Compute the alignment grid
for ti in range(len(alignments)):
  t_u = []
  for uj in range(len(alignments[ti])):
    logprob, token = alignments[ti][uj]
    token = token.to('cpu').numpy().tolist()
    decoded_token = model.decoding.decode_ids_to_tokens([token])[0] if token != model.decoding.blank_id else ''  # token at index len(vocab) == RNNT blank token
    t_u.append(decoded_token)

  print(f"Tokens at timestep {ti} = {t_u}")

Tokens at timestep 0 = ['']
Tokens at timestep 1 = ['']
Tokens at timestep 2 = ['']
Tokens at timestep 3 = ['']
Tokens at timestep 4 = ['']
Tokens at timestep 5 = ['']
Tokens at timestep 6 = ['']
Tokens at timestep 7 = ['']
Tokens at timestep 8 = ['']
Tokens at timestep 9 = ['']
Tokens at timestep 10 = ['']
Tokens at timestep 11 = ['']
Tokens at timestep 12 = ['']
Tokens at timestep 13 = ['']
Tokens at timestep 14 = ['']
Tokens at timestep 15 = ['']
Tokens at timestep 16 = ['']
Tokens at timestep 17 = ['']
Tokens at timestep 18 = ['']
Tokens at timestep 19 = ['']
Tokens at timestep 20 = ['']
Tokens at timestep 21 = ['']
Tokens at timestep 22 = ['']
Tokens at timestep 23 = ['']
Tokens at timestep 24 = ['']
Tokens at timestep 25 = ['']
Tokens at timestep 26 = ['']
Tokens at timestep 27 = ['']
Tokens at timestep 28 = ['']
Tokens at timestep 29 = ['']
Tokens at timestep 30 = ['']
Tokens at timestep 31 = ['']
Tokens at timestep 32 = ['']
Tokens at timestep 33 = ['']
Tokens at timestep 34 = 

# Takeaways
------

We covered significant ground in this tutorial, but here are a few key takeaways -

1) **Any CTC config can be easily converted to a Transducer config** by copy-pasting the default Transducer config components.

-------

2) **Dataset processing for CTC and Transducer models are the same!** If it works for CTC it works exactly the same way for Transducers. This also applies to Character-based models vs. Sub-word tokenization-based models.

-------

3) **Fused Batch during training and evaluation significantly reduces the need for large GPU memory**, and we can simply reduce the `fused_batch_size` to process samples at a larger acoustic (global) batch size (the maximum batch size that can be passed through the acoustic model, disregarding the Prediction and Joint models).

-------

4) Once trained, **transducer models implicitly can be used for beam search**! The prediction network acts as an implicit language model and can easily be used for this beam search by switching the decoding configuration.

-------

5) **Alignments can be easily extracted from Transducers** by adding a special flag to the Decoding setup. After this, it is quite simple to extract the 2-d alignment grid of the transducer for samples or a batch of samples. It must be noted that this alignment information does require slightly more memory, and increases decoding time by a small amount as well.

-------

This is just the start of our journey into Transducers. There are several improvements coming out every few months for these models, and we encourage contributions to improve the transducer framework in NeMo!

# Next Steps

Head on over to the `examples/asr` directory in the NeMo repository in order to find the training scripts for Transducer based models - `sppech_to_text_rnnt.py` (for Character decoding based Transducer models) and `speech_to_text_rnnt_bpe.py` (for Sub-word decoding based Transducer models).

You will find that following many of the steps from CTC models, and simply modifying the config to include the Transducer components, we can train character or sub-word based transducer models with the same flexibility as we can train CTC models !